In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm
import pandas as pd
from data_sources import process_parallel, transform_csv_row

In [4]:
from collections import Counter

## Encode music21 stream to text representation 

This notebook uses a full component format 
- measure separators, instruments, and separated octaves
- Format: note, octave, action type, instrument
- note repr: nG# o4 t1 i0

### Load midi data

In [6]:
version = 'v6'
data_path = Path('data/midi')
version_path = data_path/version

In [7]:
source_dir = 'midi_npz'

out_dir = 'midi_transcribe/shortdur'
source_csv = version_path/source_dir/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'midi_transcribe.csv'

In [10]:
df = pd.read_csv(out_csv); df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_transcribe/shortdur
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_transcribe/shortdur/hooktheory/pianoroll/...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_transcribe/shortdur/hooktheory/pianoroll/...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0,midi_transcribe/shortdur/hooktheory/pianoroll/...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0,midi_transcribe/shortdur/hooktheory/pianoroll/...


In [12]:
df_filtered = df.loc[df[out_dir].notna()]; df_filtered.shape

(36847, 26)

In [19]:
outliers = []
for idx,row in df_filtered.iterrows():
    with open(version_path/row[out_dir], 'r') as f:
        text = f.read()
        words = text.split(' ')
#         print(words)
        for idx,w in enumerate(words):
            if 't' in w:
                if int(w[1:]) > 200:
                    outliers.append(row[out_dir])
                    print(row[out_dir], w, words[idx-1])
                    break

midi_transcribe/shortdur/freemidi/genre-dance-eletric/Modern Talking - Cant Let You Go.txt t245 nB3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Radiohead - Bones.txt t204 nD2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.txt t664 nC5
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Radiohead - High And Dry.txt t430 nC5
midi_transcribe/shortdur/freemidi/genre-dance-eletric/C And C Music Factory - Everybody Dance Now.txt t477 nB3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Prodigy - Youll Be Under My Wheels.txt t529 nG#4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Whitney Houston - I'm Every Woman.txt t403 nB4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Gloria Estefan - Everlasting Love.txt t266 nC5
midi_transcribe/shortdur/freemidi/genre-dance-eletric/David Guetta - Baby When The Light.txt t482 nE2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Whitney Houston - Saving All My Love For Yo

midi_transcribe/shortdur/freemidi/genre-dance-eletric/Eurythmics - No More I Love You.txt t240 nG4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Donna Summer - On The Radio.txt t522 nD4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Crazy Frog - Axel F.txt t368 nC4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Take That - Babe.txt t226 nD7
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Robbie Williams - Morning Sun.txt t214 nF#2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Modern Talking - China In Her Eyes 2.txt t331 nB-4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Radiohead - Electioneering.txt t884 nA1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/2 Unlimited - No Limits.txt t216 nB5
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Alex Party - Don't Give Me Your Life.txt t452 nC4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Human League - Human.txt t647 nB-5
midi_transcribe/shortdur/freemidi/genre-dance-eletric/

midi_transcribe/shortdur/freemidi/genre-dance-eletric/David Guetta - When Love Takes Over feat. Kelly Rowland.txt t208 nG1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Alanis Morissette - The Couch.txt t225 nF2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Enrique Iglesias - Divine.txt t286 nF4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Alanis Morissette - Right Through You.txt t202 nG1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/LMFAO - Party Rock Anthem.txt t473 nG3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Human League - Love Action.txt t274 nC6
midi_transcribe/shortdur/freemidi/genre-dance-eletric/New Order - Touched By The Hand Of God.txt t264 nA4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Alanis Morissette - Front Row.txt t256 nG3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Tatu - Not Gonna Get Us.txt t319 nB2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Madonna - Bedtime Story.txt t390 nA3
midi_trans

midi_transcribe/shortdur/freemidi/genre-dance-eletric/Yellow Magic Orchestra - Waterford.txt t256 nA3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Radiohead - Bangers And Mash.txt t512 nC4
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Everything But The Girl - Missing.txt t307 nD1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Akcent - Pragnienie Milosci.txt t528 nF1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Owl City - Shooting Star.txt t231 nD2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Enrique Iglesias - Not In Love.txt t237 nG3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Eurythmics - Missionary Man.txt t1296 nA3
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Patti Austin - All Behind Us Now.txt t472 nF#1
midi_transcribe/shortdur/freemidi/genre-dance-eletric/Robbie Williams - Supreme.txt t400 nB2
midi_transcribe/shortdur/freemidi/genre-dance-eletric/New Order - World In Motion.txt t504 nF3
midi_transcribe/shortdur/freemi

midi_transcribe/shortdur/freemidi/genre-pop/Spice Girls - My Strongest Suit.txt t736 nB3
midi_transcribe/shortdur/freemidi/genre-pop/David Bowie - New Killer Star.txt t297 nB-2
midi_transcribe/shortdur/freemidi/genre-pop/Brandy - Best Friend.txt t712 nC#4
midi_transcribe/shortdur/freemidi/genre-pop/Michael Bolton - Said I Loved You But I Lied.txt t223 nG1
midi_transcribe/shortdur/freemidi/genre-pop/Enrique Iglesias - Sad Eyes.txt t208 nC#3
midi_transcribe/shortdur/freemidi/genre-pop/Vengaboys - Up And Down.txt t408 nC#6
midi_transcribe/shortdur/freemidi/genre-pop/Blondie - Picture This (2).txt t720 nF#3
midi_transcribe/shortdur/freemidi/genre-pop/Scorpions - Coast To Coast.txt t356 nA4
midi_transcribe/shortdur/freemidi/genre-pop/Phil Collins - In the Air Tonight.txt t415 nA0
midi_transcribe/shortdur/freemidi/genre-pop/Chris De Burgh - High on Emotion.txt t448 nE-2
midi_transcribe/shortdur/freemidi/genre-pop/Gloria Estefan - Everlasting Love.txt t266 nC5
midi_transcribe/shortdur/freemid

...

midi_transcribe/shortdur/from_mxl/wikifonia/Cole Porter - Well, Did You Evah?.txt t246 nD2
midi_transcribe/shortdur/from_mxl/wikifonia/Ross Parker - Monday Tuesday Wednesday.txt t208 nB3
midi_transcribe/shortdur/from_mxl/wikifonia/Frank Henry Loesser - What Are You Doing New Year's Eve?.txt t216 nC#2
midi_transcribe/shortdur/from_mxl/wikifonia/Bernie Miller - Bernie's Tune.txt t290 nA1
midi_transcribe/shortdur/from_mxl/wikifonia/Carey Landry - Only A Shadow.txt t256 nB3
midi_transcribe/shortdur/from_mxl/wikifonia/Theo Mackeben - Bei Dir War Es Immer So Schon.txt t264 nC3
midi_transcribe/shortdur/from_mxl/wikifonia/Jerry Herman - We Need A Little Christmas.txt t328 nC#4
midi_transcribe/shortdur/from_mxl/wikifonia/Frank Bjorn, Jack Harlen - ALLEY CAT SONG.txt t224 nC3
midi_transcribe/shortdur/from_mxl/wikifonia/Mel Torme, Robert Wells - The Christmas Song.txt t396 nG#3
midi_transcribe/shortdur/from_mxl/wikifonia/Barry McGuire, Randy Sparks - Green Green.txt t304 nF3
midi_transcribe/short

midi_transcribe/shortdur/from_mxl/wikifonia/Johnny Nash - I Can See Clearly Now.txt t320 nG2
midi_transcribe/shortdur/from_mxl/wikifonia/Jerome Kern, Oscar Hammerstein II - The Folks Who Live on the Hill.txt t288 nB-2
midi_transcribe/shortdur/from_mxl/wikifonia/Doc Pomus, Mort Shuman - Surrender.txt t384 nE-3
midi_transcribe/shortdur/from_mxl/wikifonia/Jimmy Henderson, Bertha Scott, Sid Robbin  - I Miss You So .txt t256 nF4
midi_transcribe/shortdur/from_mxl/wikifonia/Alexandre Morelli - Espoirs Perdus.txt t264 nC#3
midi_transcribe/shortdur/from_mxl/wikifonia/Percy Wenrich, Edward Madden - Moonlight Bay.txt t384 nE-3
midi_transcribe/shortdur/from_mxl/wikifonia/Giovanni Capurro, Eduardo di Capua - O SOLE MIO.txt t272 nG2
midi_transcribe/shortdur/from_mxl/wikifonia/Horst Jankowsky, Kal Mann - A Walk in the Black Forest.txt t400 nB-2
midi_transcribe/shortdur/from_mxl/wikifonia/John Barry, Anthony Newly, Leskie Bricusse - Goldfinger.txt t232 nG#2
midi_transcribe/shortdur/from_mxl/wikifonia/

midi_transcribe/shortdur/from_mxl/wikifonia/Michael Leonard, Herbert Martin - I'm All Smiles.txt t276 nF2
midi_transcribe/shortdur/from_mxl/wikifonia/Hoagy Carmichael, Jack Brooks - Ole Buttermilk Sky.txt t344 nD2
midi_transcribe/shortdur/from_mxl/wikifonia/Jerome Kern, Johnny Mercer - I'm Old Fashioned.txt t208 nC2
midi_transcribe/shortdur/from_mxl/wikifonia/Theodore F.Morse, Edward Madden - Two Little Boys.txt t262 nG#3
midi_transcribe/shortdur/from_mxl/wikifonia/George Cory, Nicholas Cross - I Left My Heart in San Francisco.txt t252 nB-3
midi_transcribe/shortdur/from_mxl/wikifonia/Jerry Herman - Mame.txt t216 nG#3
midi_transcribe/shortdur/from_mxl/wikifonia/Walter Marks - I've Gotta Be Me.txt t306 nF2
midi_transcribe/shortdur/from_mxl/wikifonia/Bill Trader, Bill Trader 1952 - (Now And Then There's) A  fool such as I .txt t272 nC#3
midi_transcribe/shortdur/from_mxl/wikifonia/Jerome Kern, Dorothy Fields - Pick Yourself Up.txt t416 nG1
midi_transcribe/shortdur/from_mxl/wikifonia/Vivian

midi_transcribe/shortdur/from_mxl/wikifonia/unknown - Martha Polka.txt t240 nA3
midi_transcribe/shortdur/from_mxl/wikifonia/Max Martin, Rami Yacoub - Overprotected.txt t208 nE2
midi_transcribe/shortdur/from_mxl/wikifonia/Pilat & M.Panzeri, Barry Mason(Eng.), D.Pace(orig.) - Love Me Tonight.txt t384 nG3
midi_transcribe/shortdur/from_mxl/wikifonia/Lady Gaga - Poker Face.txt t400 nA3
midi_transcribe/shortdur/from_mxl/wikifonia/Moose Charlap, Jack Gold - My Favorite Song.txt t256 nC#4
midi_transcribe/shortdur/from_mxl/wikifonia/Peter DeRose, Billy Hill - On A Little Street In Singapore.txt t216 nG2
midi_transcribe/shortdur/from_mxl/wikifonia/Clarence Gaskill, Russ Colombo,  Leo Robin - Prisoner Of Love.txt t260 nF#2
midi_transcribe/shortdur/from_mxl/wikifonia/Charles N. Daniels, Seymour Rice, Albert H. Brown - You Tell Me Your Dream.txt t380 nE-4
midi_transcribe/shortdur/from_mxl/wikifonia/George Williams, Bill Jackson and Roy Straigis - So Much In Love (Transcribed).txt t288 nF3
midi_tran

midi_transcribe/shortdur/from_mxl/wikifonia/Sammy Kahn, Jule Styne - Time after time.txt t376 nB-2
midi_transcribe/shortdur/from_mxl/wikifonia/Edward Farley & Michael Riley, Red Hodgson - The Music Goes 'Round and Around.txt t366 nG#3
midi_transcribe/shortdur/from_mxl/wikifonia/Jerry Livingston, A. J. Neiburg, Marty Symes - Under A Blanket Of Blue.txt t224 nE-3
midi_transcribe/shortdur/from_mxl/wikifonia/Hubert Giraud, Jaen Drejac, Kim Gannon - Sous le ciel de Paris (under Paris skies).txt t240 nG#3
midi_transcribe/shortdur/from_mxl/wikifonia/Thelonious Monk - Pannonica.txt t256 nF2
midi_transcribe/shortdur/from_mxl/wikifonia/ Zac Maloy, Chris Daughtry, Howard Benson - Used To.txt t234 nB3
midi_transcribe/shortdur/from_mxl/wikifonia/Richard Rodgers, Oscar Hammerstein II - It might as well be Spring.txt t264 nC#4
midi_transcribe/shortdur/from_mxl/wikifonia/George M. Cohan - Give My Regards To Broadway.txt t256 nC#4
midi_transcribe/shortdur/from_mxl/wikifonia/Antoine Domino & Dave Bartho

In [23]:
ftxt = 'midi_transcribe/shortdur/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.txt'
with open(version_path/ftxt, 'r') as f:
    text = f.read()

In [35]:
# fmd = version_path/'midi_sources/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.mid'
fmd = version_path/'midi_transform/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.mid'

In [36]:
s = file2stream(fmd)

In [37]:
print_stream_durations(s, 50)

166.0 102.0 <music21.note.Note C>
64.0 104.0 <music21.note.Note G>
132.0 204.0 <music21.note.Note G#>
116.0 270.0 <music21.note.Note C>
64.0 400.0 <music21.note.Note G#>


In [29]:
s.show('text')

{0.0} <music21.stream.Part 0x7fd69632d358>
    {0.0} <music21.instrument.Flute Flute>
    {0.0} <music21.tempo.MetronomeMark Quarter=114.0>
    {0.0} <music21.key.Key of C major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x7fd698c6c160>
        {0.0} <music21.note.Rest rest>
        {9.0} <music21.note.Note G>
        {10.0} <music21.note.Note C>
        {10.5} <music21.note.Note A>
        {11.0} <music21.note.Note G>
        {14.0} <music21.note.Note C>
        {15.3333} <music21.note.Note E>
        {17.0} <music21.note.Note C>
        {17.5} <music21.note.Note C>
        {25.0} <music21.note.Note A>
        {25.5} <music21.note.Note A>
        {26.0} <music21.note.Note B>
        {26.75} <music21.note.Note G>
        {27.3333} <music21.note.Note B>
        {30.5} <music21.note.Note A>
        {31.5} <music21.note.Note E>
        {32.6667} <music21.note.Note G>
        {41.3333} <music21.note.Note G>
        {42.0} <music21.note.Note C>
        {42.

In [24]:
text

'|| t8 nG2 t56 nD3 t7 || t1 nE3 t2 || t1 nG3 t3 || t1 nD2 t2 || t1 nG1 t29 nD3 t6 nG3 t6 nB3 t6 || t3 nD4 t2 || t3 nD3 t6 nG3 t6 nB3 t6 nG4 t2 || t1 nE4 t2 || t1 nD4 t2 || t0 nG4 t7 || t2 nD3 t6 nG3 t6 nB3 t6 || t7 nD3 t6 nG3 t6 nB3 t6 nG4 t3 || t1 nD4 t3 || t2 nB3 t3 || t0 nG1 t1 || t1 nG1 t16 nE3 t6 nG3 t6 nC4 t5 nC4 t6 || t3 nG4 t1 || t1 nG4 t10 || t1 nE3 t6 nG3 t6 nC4 t6 || t7 nE2 t8 nC4 t2 || t1 nE3 t5 || t1 nG3 t2 || t1 nE4 t3 || t1 nB1 t8 nD4 t2 || t1 nG3 t20 || t3 nG4 t2 || t1 nA1 t16 nA2 t16 nC4 t7 nE4 t7 || t3 nE4 t1 nF#4 t2 || t1 nE4 t2 || t1 nC4 t6 nE4 t6 nF#4 t2 || t0 nE4 t2 || t1 nD4 t2 || t1 nF#4 t5 || t2 nD2 t15 nC3 t6 nE3 t6 nG3 t6 || t1 nD4 t7 || t5 nC3 t6 nE3 t7 nG3 t7 || t1 nE4 t3 || t1 nC4 t2 || t1 nB3 t4 || t1 nG1 t31 nG2 t23 nD3 t6 nG3 t6 nB3 t6 || t2 nD4 t13 || t4 nD3 t8 nG3 t8 nB3 t8 || t11 nE3 t4 nG3 t4 nC4 t4 || t3 nG2 t29 || t1 nE3 t5 nG3 t4 nC4 t4 || t3 nD2 t2 || t1 nG1 t20 nD3 t6 nG3 t6 nB3 t6 || t4 nD4 t2 || t2 nD3 t6 nG3 t6 nB3 t6 nG4 t2 || t0 nE4 t2 || 

In [14]:
transcribed_files = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

In [15]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=list(transcribed_files.keys()))

In [16]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

((36847, 1), (39942, 25), (39942, 26))

In [17]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform,midi_npz,midi_npz_timesteps,midi_transcribe/shortcont
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,145.0,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...,midi_npz/hooktheory/pianoroll/w/wayne-sharpe/y...,129.0,midi_transcribe/shortcont/hooktheory/pianoroll...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...,midi_npz/hooktheory/pianoroll/w/what-a-day/kie...,65.0,midi_transcribe/shortcont/hooktheory/pianoroll...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...,midi_npz/hooktheory/pianoroll/w/whiteflame/sen...,129.0,midi_transcribe/shortcont/hooktheory/pianoroll...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...,midi_npz/hooktheory/pianoroll/w/wham/last-chri...,129.0,midi_transcribe/shortcont/hooktheory/pianoroll...
